# MRI Classification


In [17]:
## Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models,datasets,layers
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import imutils
import os
from os import listdir
import SimpleITK
from imutils import paths
import cv2 as cv
import numpy as np
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
##image = cv.imread(directory + "\ " + filename)
%matplotlib inline

## Preprocessing

In [11]:
def crop_brain_contour(img):
    def ShowImage(title,img):
        plt.figure(figsize=(10, 10))
        plt.axis('Off')
        plt.imshow(img,cmap='gray')
        plt.title(title)
        plt.show()
    

    org_img = img
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(gray,0,255,cv.THRESH_OTSU)
    ret, markers = cv.connectedComponents(thresh)
    #Get the area taken by each component. Ignore label 0 since this is the background.
    marker_area = [np.sum(markers==m) for m in range(np.max(markers)) if m!=0] 
    #Get label of largest component by area
    largest_component = np.argmax(marker_area)+1 #Add 1 since we dropped zero above                        
    #Get pixels which correspond to the brain
    brain_mask = markers==largest_component

    img = org_img
    brain_out = img.copy()
    #In a copy of the original image, clear those pixels that don't correspond to the brain
    brain_out[brain_mask==False] = (0,0,0)

    img = org_img
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)


    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)

    # sure background area
    sure_bg = cv.dilate(opening,kernel,iterations=3)

    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)

    # Marker labelling
    ret, markers = cv.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1


    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    markers = cv.watershed(img,markers)
    img[markers == -1] = [255,0,0]

    im1 = cv.cvtColor(img,cv.COLOR_HSV2RGB)
    
    
    blur1 = cv.bilateralFilter(gray,9,75,75)
    #gaussian filter
    blur2 = cv.GaussianBlur(gray,(5,5),0)
    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv.erode(thresh, None, iterations=2)


    # Find contours in thresholded image, then grab the largest one
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv.contourArea)
    
    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = im1[extTop[1]:extBot[1], extLeft[0]:extRight[0]]  
    #### ShowImage('Water_shed_cropped image',new_image) ####
    return new_image

In [12]:
X=[]     ## Images    
Y=[]     ## Labels 
images_yes = list(paths.list_images('/Users/apple/Desktop/Image processing/archive/yes'))
for i in images_yes:
    # load the image
        image = cv.imread(i)
        # crop the brain and ignore the unnecessary rest part of the image
        image = crop_brain_contour(image)
        # resize image
        image = cv.resize(image, dsize=(240, 240), interpolation=cv.INTER_CUBIC)
        # normalize values
        image = image / 255.
        # convert image to numpy array and append it to X
        X.append(image)
        Y.append(1)
        ########## We have here X --> images of tumour vala brain Y--> all 1's
images_no = list(paths.list_images('/Users/apple/Desktop/Image processing/archive/no'))
for i in images_no:
    # load the image
        image = cv.imread(i)
        # crop the brain and ignore the unnecessary rest part of the image
        image = crop_brain_contour(image)
        # resize image
        image = cv.resize(image, dsize=(240, 240), interpolation=cv.INTER_CUBIC)
        # normalize values
        image = image / 255.
        # convert image to numpy array and append it to X
        X.append(image)
        Y.append(0)
X = np.array(X)
## def func(X):
 #   for each element in X:
 #       X_segment.append(Segemennted (x))
        
Y = np.array(Y)
# Shuffle the data
X, Y = shuffle(X, Y)    
## printing an image



### Randomly Splitting the data into test and train sets

In [13]:
## Data loading
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
## Data normalising
X_train,X_test = X_train/255,X_test/255
print(X_train.shape)
print(Y_train.shape)

(169, 240, 240, 3)
(169,)


###  Neural Network

In [22]:
## Neural Network 
model = models.Sequential()
## 1st Layer is convolutional layer with relu activation function
model.add(layers.Conv2D(32,(3,3),activation ='relu',input_shape=(240,240,3)))
## 2nd Layer is Maxpooling layer for the convolutional layer
model.add(layers.MaxPooling2D((2,2)))
## 3rd and 4th layer are same as above. 4th layer has 32 units
model.add(layers.Conv2D(64,(3,3),activation = 'relu' ))
model.add(layers.MaxPooling2D((2,2)))
## 5th layer is another convolutional layer with 32 units
model.add(layers.Conv2D(64,(3,3),activation = 'relu' ))
##6th layer is Flattening layee
model.add(layers.Flatten())
##7th layer is a dense layer with 64 units with relu activation function
model.add(layers.Dense(units= 64,activation='relu'))
##7th layer is a dense layer with 64 units with relu activation function
model.add(layers.Dense(units= 32,activation='relu'))
## 8th and the final layer is another dense layer with softmax function with 2 units
model.add(layers.Dense(units= 2,activation='sigmoid'))  

In [23]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'],)
model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Epoch 1/10
6/6 [==============================] - 17s 3s/step - loss: 0.6679 - accuracy: 0.5858 - val_loss: 0.6855 - val_accuracy: 0.5714
Epoch 2/10
6/6 [==============================] - 15s 3s/step - loss: 0.6667 - accuracy: 0.6331 - val_loss: 0.6827 - val_accuracy: 0.5714
Epoch 3/10
6/6 [==============================] - 13s 2s/step - loss: 0.6619 - accuracy: 0.6331 - val_loss: 0.7005 - val_accuracy: 0.5714
Epoch 4/10
6/6 [==============================] - 13s 2s/step - loss: 0.6623 - accuracy: 0.6331 - val_loss: 0.6917 - val_accuracy: 0.5714
Epoch 5/10
6/6 [==============================] - 17s 3s/step - loss: 0.6591 - accuracy: 0.6331 - val_loss: 0.6888 - val_accuracy: 0.5714
Epoch 6/10
6/6 [==============================] - 24s 4s/step - loss: 0.6566 - accuracy: 0.6331 - val_loss: 0.6849 - val_accuracy: 0.5714
Epoch 7/10
6/6 [==============================] - 26s 4s/step - loss: 0.6598 - accuracy: 0.6331 - val_loss: 0.6876 - val_accuracy: 0.5714
Epoch 8/10
6/6 [==================

In [25]:
## Loss and Accuracy of the model
loss,accuracy = model.evaluate(X_test,Y_test)

print(str(loss) + ' is the loss')
print(str(accuracy) + ' is the accuracy')



3/3 [==============================] - 2s 663ms/step - loss: 0.6939 - accuracy: 0.5714
0.6939208507537842 is the loss
0.5714285969734192 is the accuracy


In [26]:
model.save('MRI_image_class_V2.model')

INFO:tensorflow:Assets written to: MRI_image_class_V2.model/assets
